# Dataset Exploration

In [21]:
import json
import pandas as pd

In [22]:
# path to data
headqa_paths = ["data_raw/HEAD/HEAD/test_HEAD.json", "data_raw/HEAD/HEAD/train_HEAD.json"] # path to labeled data
cowese_path = "data_raw/CoWeSe.txt" # path to unlabeled data

In [23]:
cowese_data = []
with open(cowese_path, 'r', encoding='utf-8') as f:
    for line in f:
        cowese_data.append(line.strip())

print(f'Number of questions in CoWeSe: {len(cowese_data)}')

Number of questions in CoWeSe: 40918081


In [24]:
# read headqa data
headqa_data = []

for path in headqa_paths:
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        cuadernos = list(data['exams'].keys())
        for cuaderno in cuadernos:
            # if cuaderno[-1] == 'M' or cuaderno[-1] == 'F' or cuaderno[-1] == 'E': # only keep medical exams
            questions = data['exams'][cuaderno]['data']
            headqa_data.extend(questions)

# eliminate questions with an image
headqa_data = [question for question in headqa_data if question['image'] == '']
print(f'Number of questions in HeadQA: {len(headqa_data)}')

Number of questions in HeadQA: 5264


In [25]:
headqa_data[1000]

{'qid': '127',
 'qtext': '¿Cuál de las siguientes aportaciones teóricas en la comprensión de la depresión hipotetiza que la misma se debe a déficits específicos en la autoobservación, la autoevaluación y el autorrefuerzo?:',
 'ra': '4',
 'answers': [{'aid': 1, 'atext': 'Enfoque comportamental de Lewinsohn.'},
  {'aid': 2, 'atext': 'Enfoque interpersonal de Coyne.'},
  {'aid': 3, 'atext': 'Enfoque procesal sistémico de Guidano.'},
  {'aid': 4, 'atext': 'Enfoque de autocontrol de Rehm.'}],
 'image': ''}

Exploring balance in prediction classes (letter of correct answer)

In [26]:
max_num_answers = set([len(question['answers']) for question in headqa_data])
count_num_answers = {i:0 for i in max_num_answers}
count_correct_answers = {i:0 for i in range(5)}

for question in headqa_data:
    count_correct_answers[int(question['ra'])-1] += 1
    count_num_answers[len(question['answers'])] += 1

print(f'correct answers {count_correct_answers}')
print(count_num_answers)

correct answers {0: 1183, 1: 1264, 2: 1256, 3: 1156, 4: 405}
{4: 2675, 5: 2589}


## Padding dummy answers
Notice that some questions have 4 possible answers, while some others have 5. We want to pad the answers so that the output results are always 5. For that reason we add a dummy answer to the questions with a missing possible answer, with semantic meaning to not confuse the model.

In [27]:
changed_questions = 0
for question in headqa_data:
    if len(question['answers']) == 4:
        question['answers'].append({
            'aid':5,
            'atext': 'Respuesta incorrecta',
        })

        changed_questions += 1
        
print(headqa_data[1000])

{'qid': '127', 'qtext': '¿Cuál de las siguientes aportaciones teóricas en la comprensión de la depresión hipotetiza que la misma se debe a déficits específicos en la autoobservación, la autoevaluación y el autorrefuerzo?:', 'ra': '4', 'answers': [{'aid': 1, 'atext': 'Enfoque comportamental de Lewinsohn.'}, {'aid': 2, 'atext': 'Enfoque interpersonal de Coyne.'}, {'aid': 3, 'atext': 'Enfoque procesal sistémico de Guidano.'}, {'aid': 4, 'atext': 'Enfoque de autocontrol de Rehm.'}, {'aid': 5, 'atext': 'Respuesta incorrecta'}], 'image': ''}


## Saving data into a dataframe

In [28]:
rows = []
for question in headqa_data:
    row = {
        'qid': question['qid'],
        'qtext': question['qtext'],
        'ra': int(question['ra']),
        'image': question['image']
    }

    # Agregar las respuestas como columnas separadas
    for ans in question['answers']:
        row[f'answer_{ans["aid"]}'] = ans['atext']

    rows.append(row)

# Crear el DataFrame
df = pd.DataFrame(rows)
df.to_csv('data/head_qa_dataset.csv')

In [29]:
df.head()

,qid,qtext,ra,image,answer_1,answer_2,answer_3,answer_4,answer_5
0,1,Forma fibras extracelulares con gran resistenc...,2,,Fibronectina.,Colágeno.,Integrinas.,Proteoglucanos.,Respuesta incorrecta
1,2,El fosfolípido cardiolipina es abundante en la...,1,,Mitocondrial interna.,Mitocondrial externa.,Plasmática.,Lisosomal.,Respuesta incorrecta
2,3,NO es una función de los filamentos intermedios:,4,,Proporcionar apoyo estructural a la célula.,Fijar el núcleo en su sitio.,Proveer resistencia mecánica.,Locomoción celular.,Respuesta incorrecta
3,4,Los cuerpos multivesiculares son:,4,,Peroxisomas.,Mitocondrias.,Polisomas.,Endosomas.,Respuesta incorrecta
4,5,Forman la envuelta de mielina de los axones en...,2,,Oligodendrocitos.,Células de Schwann.,Células microgliales.,Células ganglionares.,Respuesta incorrecta


## Save CoWeSe data into documents

In [30]:
# create complete texts instead of just sentences
complete_documents = []
complete_document = ""
for text in cowese_data:
    if text == "":
        complete_documents.append(complete_document)
        complete_document = ""
    else:
        complete_document += text + " "

# remove trailing spaces
complete_documents = [document.strip() for document in complete_documents]

print(len(complete_documents))

1973048


In [31]:
complete_documents[100]

'El principio de la antibioticoprofilaxis consiste en prevenir la aparición de una enfermedad infecciosa mediante la erradicación de una población patógena ya presente en el organismo o impidiendo la implantación de una población bacteriana exógena. Palabras Clave: Infección bacteriana, Antibioticoprofilaxis, Paciente de riesgo, Contagio, Infecciones recidivantes'

In [32]:
complete_documents[101]

'Las micosis superficiales son infecciones cutáneas frecuentes. No causan signos sistémicos y, aunque su diagnóstico es ante todo clínico, es fundamental obtener una muestra para el estudio micológico para confirmar el diagnóstico y orientar la conducta terapéutica. El tratamiento se basa en el uso de antifúngicos tópicos, y no tanto en los antimicóticos sistémicos. La evolución a veces está marcada por las recidivas. Las dermatofitosis son las micosis cutáneas más frecuentes y pueden localizarse en el cuero cabelludo (tiñas tonsurantes, tiñas supuradas y tiñas fávicas), los pliegues (intertrigo dermatofítico), la piel lampiña o las uñas. La contaminación es de origen humano (dermatofitos antropófilos), animal (zoófilos) o telúrico (geófilos). En cuanto a las afecciones del cuero cabelludo, la exploración con luz de Wood orienta el diagnóstico y guía la obtención de la muestra para el estudio micológico. Los antifúngicos tópicos deben aplicarse de manera prolongada y se recetan por vía

In [33]:
complete_documents[0]

'Existen diversos factores que influyen en el paso de sustancias a la leche materna como, por ejemplo, la unión a proteínas plasmáticas, ionización, grado de liposolubilidad, peso molecular, etc. Tales parámetros varían según los fármacos. Además, no hay suficientes estudios para un número elevado de medicamentos, sobre todo para los de reciente comercialización. De hecho, la recomendación «contraindicado durante la lactancia», hace referencia sobre todo a la falta de estudios farmacocinéticos sobre la excreción en leche materna, y no a la existencia de observaciones clínicas. En este artículo se exponen los datos disponibles sobre fármacos de uso común y algunos principios sencillos para facilitar la labor de los profesionales de la salud a la hora de prescribir medicación a una mujer durante la lactancia.'

In [ ]:
df_cowese = pd.DataFrame({'document': complete_documents})

# Guardar a CSV
df_cowese.to_csv("data/documentos_cowese.csv", index=False, encoding="utf-8")

In [35]:
df_cowese.head()

,document
0,Existen diversos factores que influyen en el p...
1,El asma es una enfermedad inflamatoria crónica...
2,Las dilataciones bronquiales o bronquiectasias...
3,La tuberculosis es una enfermedad infecciosa q...
4,El tratamiento de las disfunciones eréctiles h...
